In [ ]:
!pip install -U gpboost # 0.7.9
!pip install -U seaborn
!pip install scikit-learn==1.0.2
!pip install pdpbox

In [ ]:
import glob
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV
from preprocessing.outlier_methods import detect_outliers, plot_distribution
import gpboost as gpb
from rfe_gpboost import *
from gpboost_utils import *
warnings.filterwarnings("ignore")

%matplotlib inline

## Read and preprocess data

In [ ]:
df_long = pd.read_csv("path/to/file.csv")

In [ ]:
###########################
# Remove other cancer cases
###########################
print(f"Number of participants: {len(df_long.eid.unique())}")
df_label = pd.read_csv("path/to/file.csv")

controls_with_othercancer = (
    pd.merge(df_long[["eid"]], df_label, on="eid", how="left")
    .query("label_first_occurred_date.isna() & othercancer_first_occurred_date.notna()", engine="python")
    .eid.unique()
)
print(f"Number of participants who developed cancer other than CRC: {len(controls_with_othercancer)}")
df_filtered = df_long.loc[~df_long.eid.isin(controls_with_othercancer), :]

othercancer_pre_crc = (
    pd.merge(df_long[["eid"]], df_label, on="eid", how="left")
    .query("(label_first_occurred_date.notna()) & (othercancer_first_occurred_date.notna()) & (label_first_occurred_date>othercancer_first_occurred_date)", engine="python")
    .eid.unique()
)
print(f"Number of participants who developed other cancer prior to CRC: {len(othercancer_pre_crc)}")
df_filtered = df_filtered.loc[~df_filtered.eid.isin(othercancer_pre_crc), :]

othercancer_with_crc = (
    pd.merge(df_long[["eid"]], df_label, on="eid", how="left")
    .query("(label_first_occurred_date.notna()) & (othercancer_first_occurred_date.notna()) & (label_first_occurred_date==othercancer_first_occurred_date)", engine="python")
    .eid.unique()
)
print(f"Number of participants who developed other cancer same time as CRC: {len(othercancer_with_crc)}")
df_filtered = df_filtered.loc[~df_filtered.eid.isin(othercancer_with_crc), :]

print(f"Number of participants left: {len(df_filtered.eid.unique())}")

In [ ]:
###########################
# Select features
###########################
selected_cols = ["age", "sex", "ethnicity", "townsend", "alcohol", "smoke", "fasted", 
                 "redmeat_intake", "oily_fish_intake", "famhist_cancer", "edu_university", "regular_aspirin", "health_rating", 
                 "diseasehist_ibd", "diseasehist_cardiovascular", "diseasehist_anyliverbiliary", "diseasehist_diabetes",
                 
                 "hgrip", "tlr", "whr", "bmi", "height", "met_rate", "impedance", "sleep_dur", 
                 "sbp", "dbp", "pulse", "met_mins",
                 
                 "hgb", "hct", "wbc", "rbc", "plt", "lym", "mcv", "mono", "neut", "eos", "baso", "n_rbc", "reti",
                 "u_sodium", 'u_potas', "u_cr",
                 'apoa', 'apob',  'chol', 'hdl', 'ldl', 'tgly', 'urea', 'crp','tprotein',
                 'glu', 'phos', 'alb', 'alp', 'alt', 'ast', 'ggt', 'urate', 'd_bil', 't_bil',
                 'shbg', 'igf1', 'vitd', 'cysc', 'calc',  'hba1c', 'tst'] 

X = df_filtered.loc[:, ["eid", "is_label"] + [col for col in df_filtered.columns if col.split("-")[0] in selected_cols]]
print(X.shape)

In [ ]:
###################################
# categorize non continuous columns
###################################
### replace nan with unk
X["fasted"] = X["fasted"].astype(float)
X["ethnicity"] = X["ethnicity"].apply(lambda x: "unk" if pd.isnull(x) == True else ("white" if x == 1 else "nonwhite"))
X["met_mins"] = pd.qcut(X.loc[:, "met_mins"], q=5, labels=range(1, 6)).values.add_categories("unk").fillna("unk")

X.replace(
    {
        "redmeat_intake": {np.nan: "unk"},
        "oily_fish_intake": {np.nan: "unk"},
        "famhist_cancer": {np.nan: "unk"},
        "edu_university": {np.nan: "unk"},
        "regular_aspirin": {np.nan: "unk"},
        "health_rating": {np.nan: "unk"},
        "alcohol": {np.nan: "unk"},
        "smoke": {np.nan: "unk"},
        "diseasehist_ibd": {np.nan: "unk"},
        "diseasehist_cardiovascular": {np.nan: "unk"},
        "diseasehist_anyliverbiliary": {np.nan: "unk"},
        "diseasehist_diabetes": {np.nan: "unk"},
    },
    inplace=True,
)

X["ethnicity"] = pd.Categorical(X["ethnicity"], categories=["white", "nonwhite", "unk"])
X["redmeat_intake"] = pd.Categorical(X["redmeat_intake"], categories=[0, 1, 2, 3, 4, 5, "unk"])
X["oily_fish_intake"] = pd.Categorical(X["oily_fish_intake"], categories=[0, 1, 2, 3, 4, 5, "unk"])
X["famhist_cancer"] = pd.Categorical(X["famhist_cancer"], categories=[False, True, "unk"])
X["diseasehist_ibd"] = pd.Categorical(X["diseasehist_ibd"], categories=[False, True, "unk"])
X["diseasehist_cardiovascular"] = pd.Categorical(X["diseasehist_cardiovascular"], categories=[False, True, "unk"])
X["diseasehist_anyliverbiliary"] = pd.Categorical(X["diseasehist_anyliverbiliary"], categories=[False, True, "unk"])
X["diseasehist_diabetes"] = pd.Categorical(X["diseasehist_diabetes"], categories=[False, True, "unk"])
X["edu_university"] = pd.Categorical(X["edu_university"], categories=[False, True, "unk"])
X["regular_aspirin"] = pd.Categorical(X["regular_aspirin"], categories=[False, True, "unk"])
X["health_rating"] = pd.Categorical(X["health_rating"], categories=[4, 3, 2, 1, "unk"])  # 1-excellent, 2-good, 3-fair, 4-poor
X["alcohol"] = pd.Categorical(X["alcohol"], categories=[0, 1, 2, 3, 4, 5, 6, "unk"])
X["smoke"] = pd.Categorical(X["smoke"], categories=[0, 1, 2, 3, 4, "unk"])

categorical_features=["sex", "ethnicity", "fasted", "famhist_cancer", "edu_university", "regular_aspirin", 
                      "diseasehist_cardiovascular", "diseasehist_anyliverbiliary", "diseasehist_diabetes",
                     "smoke", "alcohol", "redmeat_intake", "oily_fish_intake", "health_rating", "met_mins"] 

In [ ]:
##################
# remove outliers
##################
continuous_vars = ["hgrip", "tlr", "whr",  "height", "met_rate", "impedance", "sleep_dur", 
                 "sbp", "dbp", "pulse", "bmi",
                 
                 "hgb", "hct", "wbc", "rbc", "plt", "lym", "mcv", "mono", "neut", "eos", "baso", "n_rbc", "reti",
                 "u_sodium", 'u_potas', "u_cr",
                 'apoa', 'apob',  'chol', 'hdl', 'ldl', 'tgly', 'urea', 'crp','tprotein',
                 'glu', 'phos', 'alb', 'alp', 'alt', 'ast', 'ggt', 'urate', 'd_bil', 't_bil',
                 'shbg', 'igf1', 'vitd', 'cysc', 'calc',  'hba1c', 'tst']

outliers = []
for i, col in enumerate(continuous_vars):
    outliers_ = detect_outliers(X, col, method="percentile", percentile_threshold=0.001)
    outliers += list(outliers_)

outliers = np.unique(outliers)
print(f"Number of outliers: {len(outliers)}")

X.drop(outliers, axis="index", inplace=True)
print(X.shape)

plot_distribution(X, continuous_vars, ncols=6, plot="histogram", figsize=(20, 20))

In [ ]:
# Number of healthy vs CRC after processing
X.groupby("eid").is_label.max().value_counts()

## Model and evaluate

In [ ]:
seed = 1
n_splits = 5
output_name = "gpboost_results"

### Prepare data

In [ ]:
train_eids, holdout_eids = split_dataset(X, seed, balance=False, pct_train=0.8)

X_kfold = X[X.eid.isin(train_eids)].reset_index(drop=True)
X_holdout = X[X.eid.isin(holdout_eids)].reset_index(drop=True)

In [ ]:
cv, cv_undersampled = stratified_group_kfold_split_undersampled(X_kfold, n_splits, seed)

In [ ]:
# Indices of all selected datapoints by undersampling
idx_us = []
for (train_idx, test_idx) in cv_undersampled:
    idx_us += list(train_idx)
    idx_us += list(test_idx)

# Get new reset indices for the undersampled data, this is necesssary to be able to use GridSearchCV
X_kfold_us = X_kfold.iloc[np.unique(idx_us), :].reset_index()
new_to_old_idx = X_kfold_us.pop("index").to_dict()
old_to_new_idx = {v: k for k, v in new_to_old_idx.items()}
y_kfold_us = X_kfold_us.pop("is_label").to_numpy()

cv_undersampled_reset = []
for (train_idx, test_idx) in cv_undersampled:
    cv_undersampled_reset.append((np.vectorize(old_to_new_idx.get)(train_idx), np.vectorize(old_to_new_idx.get)(test_idx)))

In [ ]:
# Save indices/eids for splits
with open(f"{output_name}/{n_splits}fold_cv_{seed}.npy", "wb") as f:
    np.save(f, cv, allow_pickle=True)
    np.save(f, cv_undersampled, allow_pickle=True)
    np.save(f, cv_undersampled_reset, allow_pickle=True)

with open(f"{output_name}/train_test_eids_{seed}.npy", "wb") as f:
    np.save(f, train_eids, allow_pickle=True)
    np.save(f, holdout_eids, allow_pickle=True)

In [ ]:
# Load saved data
with open(f"{output_name}/train_test_eids_{seed}.npy", "rb") as f:
    train_eids = np.load(f, allow_pickle=True)
    holdout_eids = np.load(f, allow_pickle=True)

X_kfold = X[X.eid.isin(train_eids)].reset_index(drop=True)
X_holdout = X[X.eid.isin(holdout_eids)].reset_index(drop=True)

with open(f"{output_name}/{n_splits}fold_cv_{seed}.npy", "rb") as f:
    cv = np.load(f, allow_pickle=True)
    cv_undersampled = np.load(f, allow_pickle=True)
    cv_undersampled_reset = np.load(f, allow_pickle=True)

idx_us = []
for (train_idx, test_idx) in cv_undersampled:
    idx_us += list(train_idx)
    idx_us += list(test_idx)
X_kfold_us = X_kfold.iloc[np.unique(idx_us), :].reset_index(drop=True)

### Hyperparameter optimization for gpboost

#### Imbalanced data

In [ ]:
#########################
# Hyperparam optimization
#########################
num_boost_round = 5000
params = {
    "objective": "binary",
    "metric": "auc",
    "seed": 1,
    "scale_pos_weight": sum(X_kfold["is_label"] == 0) / sum(X_kfold["is_label"]),
}
param_grid = {
    "learning_rate": [0.2, 0.1, 0.01],
    "min_data_in_leaf": [10, 50, 100],
    "max_depth": [3, 5, 10, -1],
}

opt_params = gpboost_cv_hyperparam_search(X_kfold, cv, param_grid, params, num_boost_round, categorical_features, early_stopping_rounds=10)

print("Best parameters: " + str(opt_params["best_params"]))
print("Best number of iterations: " + str(opt_params["best_iter"]))
print("Best score: " + str(opt_params["best_score"]))

In [ ]:
###################################################################
# Cross-validation for determining number of boosting iterations
###################################################################
num_boost_round = 5000
params = {
    "objective": "binary",
    "metric": "auc",
    "seed": 1,
    "scale_pos_weight": sum(X_kfold["is_label"] == 0) / sum(X_kfold["is_label"]),
    "learning_rate": 0.2,
    "max_depth": 5,
    "min_data_in_leaf": 100,
}

cvbst = gpboost_cv_niters_search(X_kfold, cv, params, num_boost_round, categorical_features, early_stopping_rounds=10)

#### Undersampled data

In [ ]:
#########################################
# Hyperparam optimization - undersampled
#########################################
num_boost_round = 5000
params = {
    "objective": "binary",
    "metric": "auc",
    "seed": 1,
}
param_grid = {
    "learning_rate": [0.2, 0.1, 0.01],
    "min_data_in_leaf": [10, 50, 100],
    "max_depth": [3, 5, 10, -1],
}

opt_params = gpboost_cv_hyperparam_search(X_kfold_us, cv_undersampled_reset, param_grid, params, num_boost_round, categorical_features, early_stopping_rounds=10)

print("Best parameters: " + str(opt_params["best_params"]))
print("Best number of iterations: " + str(opt_params["best_iter"]))
print("Best score: " + str(opt_params["best_score"]))

In [ ]:
###################################################################
# Cross-validation for determining number of boosting iterations
###################################################################
num_boost_round = 5000
params = {
    "objective": "binary",
    "metric": "auc",
    "seed": 1,
    "learning_rate": 0.1,
    "max_depth": 3,
    "min_data_in_leaf": 50,
}

cvbst = gpboost_cv_niters_search(X_kfold_us, cv_undersampled_reset, params, num_boost_round, categorical_features, early_stopping_rounds=10)

### Run RFE to find n_features_to_select

In [ ]:
# specify tree-boosting parameters as a dict
gpb_params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "verbose": 0,
    "seed": 1,
    "learning_rate": 0.1,
    "max_depth": 3,
    "min_data_in_leaf": 50,
}
num_boost_round = 90

In [ ]:
# Run RFE to select best n features
n_features_to_select = [1, 2, 3, 4, 5]
metric = "roc_auc"
importance_type = "gain"

selector = RFE_Gpboost(categorical_features, gpb_params, num_boost_round, importance_type)
gscv = GridSearchCV(
    estimator=selector,
    param_grid={"n_features_to_select": n_features_to_select},
    cv=cv_undersampled_reset,
    scoring=metric,
    refit=False,
    verbose=3,
    error_score="raise",
)
gscv.fit(X=X_kfold_us, y=y_kfold_us)

In [ ]:
selected_n_features_to_select = gscv.best_params_["n_features_to_select"]
print(f"Selected n_features_to_select: {selected_n_features_to_select}")

In [ ]:
# gscv_cv_results = gscv.cv_results_.copy()
# np.save(f'{output_name}/rfe_gscv_cv_results.npy', gscv.cv_results_)
gscv_cv_results = np.load(f"{output_name}/rfe_gscv_cv_results.npy", allow_pickle="TRUE").item()

In [ ]:
for i in range(len(gscv_cv_results["params"])):
    print(f"For n_features_to_select={gscv_cv_results['params'][i]['n_features_to_select']} \
    ROC_AUC score = {gscv_cv_results['mean_test_score'][i]:.3f} +/- {gscv_cv_results['std_test_score'][i]:.3f}")

plt.figure(figsize=(5, 4))
plt.plot(list(range(1, len(gscv_cv_results["mean_test_score"]) + 1)), gscv_cv_results["mean_test_score"], color="blue")
plt.errorbar(list(range(1, len(gscv_cv_results["mean_test_score"]) + 1)), gscv_cv_results["mean_test_score"], gscv_cv_results["std_test_score"],
    color="blue", fmt="o", capsize=3)
plt.xticks(list(range(1, len(gscv_cv_results["mean_test_score"]) + 1)))
plt.xlabel("Number of features selected")
plt.ylabel("AUC-ROC")
# plt.savefig(f"{output_name}/plots/rfe_nfeatures_rocauc.png", dpi=300, bbox_inches="tight")
plt.show()

### Aggregate feature importances from kfold experiments to select features

In [ ]:
for fold_i, (train_index, test_index) in enumerate(cv_undersampled):
    X_train_us = X_kfold.loc[train_index, :]
    X_test_us = X_kfold.loc[test_index, :]
    y_train_us = X_train_us.pop("is_label").to_numpy()
    y_test_us = X_test_us.pop("is_label").to_numpy()

    bst = run_gpboost(X_train_us, y_train_us, categorical_features, gpb_params, num_boost_round)
    importance, gpboost_results = get_model_results(bst, X_train_us, y_train_us, X_test_us, y_test_us,
        categorical_features, output_name, seed, f"{fold_i}_us")

In [ ]:
files = [f for f in sorted(glob.glob(f"{output_name}/gpboost_importance_{seed}_*_us.csv"))
        if ("_holdout" not in f) and ("_combo" not in f)]

results_gain, results_split = summarize_importance_results(files)
results_gain.sort_values("mean", ascending=False).round(3).head(10)

In [ ]:
selected_combo = results_gain.head(selected_n_features_to_select).index.tolist()
print(f"Selected combo by gain importance: {selected_combo}")

### Run with all and selected features on the whole kfold dataset

In [ ]:
gpb_params_imbalanced = {
    "objective": "binary",
    "metric": "binary_logloss",
    "verbose": 0,
    "seed": 1,
    "scale_pos_weight": sum(X_kfold["is_label"] == 0) / sum(X_kfold["is_label"]),  
    "learning_rate": 0.2,
    "min_data_in_leaf": 100,
    "max_depth": 5,
}
num_boost_round = 180

print("==== All features - train on whole k-fold ====")
X_kfold_all = X_kfold.copy(deep=True)
y_kfold_all = X_kfold_all.pop("is_label").to_numpy()
X_holdout_all = X_holdout.copy(deep=True)
y_holdout_all = X_holdout_all.pop("is_label").to_numpy()

bst = run_gpboost(X_kfold_all, y_kfold_all, categorical_features, gpb_params_imbalanced, num_boost_round)
importance, gpboost_results = get_model_results(bst, X_kfold_all, y_kfold_all, X_holdout_all, y_holdout_all, categorical_features, output_name, seed, "holdout_all")


print("==== Selected features - train on whole k-fold ====")
selected_combo = ["age", "health_rating", "alp", "wbc", "lym"]
X_kfold_combo = X_kfold.loc[:, selected_combo + ["eid", "is_label"]]
y_kfold_combo = X_kfold_combo.pop("is_label").to_numpy()
X_holdout_combo = X_holdout.loc[:, selected_combo + ["eid", "is_label"]]
y_holdout_combo = X_holdout_combo.pop("is_label").to_numpy()

categorical_features_combo = [c for c in categorical_features if c in selected_combo]
bst = run_gpboost(X_kfold_combo, y_kfold_combo, categorical_features_combo, gpb_params_imbalanced, num_boost_round)
importance, gpboost_results = get_model_results(bst, X_kfold_combo, y_kfold_combo, X_holdout_combo, y_holdout_combo, categorical_features_combo, output_name, seed, "holdout_combo")

### Holdout Set Results

In [ ]:
pred_files = [
    f"{output_name}/gpboost_test_predictions_{seed}_holdout_combo.csv",
    f"{output_name}/gpboost_test_predictions_{seed}_holdout_all.csv",
]
colors = ["red", "blue"]
scores = []
plt.figure(figsize=(5, 4))
for i, f in enumerate(pred_files, 1):
    pred_file = pd.read_csv(f)
    y_test = pred_file["y_test"].ravel()
    y_pred = pred_file["y_pred_prob"].ravel()

    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test, y_pred)
    score = roc_auc_score(y_test, y_pred)
    scores.append(score)
    sname = "selected" if i == 1 else "all"
    plt.plot(false_positive_rate, true_positive_rate,
        label=r"$AUC_{{{sname}\/features}}={{{score:.3f}}}$".format(sname=sname, score=score),
        color=colors[i - 1])

plt.title("Receiver Operating Characteristic")
plt.plot([0, 1], ls="--")
plt.ylabel("True Positive Rate \n(Sensitivity)")
plt.xlabel("False Positive Rate \n(1-Specificity)")
plt.legend()
# plt.savefig(f"{output_name}/plots/gpboost_holdout_ROC_curve.png", dpi=300, bbox_inches="tight")
plt.show()

### Plot Feature Importance

In [ ]:
import json
with open("ukb_feature_rename_map.json", "r") as f:
    rename_mapping = json.load(f)

In [ ]:
files = [f for f in sorted(glob.glob(f"{output_name}/gpboost_importance_{seed}_*_us.csv")) 
        if ("_holdout" not in f) and ("_combo" not in f)]

results_gain, results_split = summarize_importance_results(files)
results_gain.rename(index=rename_mapping, inplace=True)
results_split.rename(index=rename_mapping, inplace=True)
results_gain.round(3).head(10)

In [ ]:
plt.rcParams["axes.axisbelow"] = True

top_n = 20
plt.figure(figsize=(8, 9))
plt.rcParams.update({"font.size": 16})
plt.rc("axes", labelsize=16)
sns.boxplot(
    data=results_gain.loc[
        results_gain.sort_values("mean", ascending=False).head(top_n).index,
        [f"{seed}_{i}_us" for i in range(5)],
    ].transpose(),
    orient="h",
    whis=[0, 100],
)
plt.xlabel("Normalised feature importance (gain-based)")
plt.ylabel("")
plt.grid(color="lightgrey", linestyle="-")
# plt.savefig(f"{output_name}/plots/gpboost_kfold_feature_importance_gain_top{top_n}.png", dpi=300, bbox_inches="tight")
plt.show()

### Partial Dependence Plots

In [ ]:
bst = gpb.Booster(model_file=f"{output_name}/gpboost_model_{seed}_holdout_combo.json")

In [ ]:
feature_to_plot = "age"
feature_description = "Age"
plot_params = {"title": "Partial dependency of CRC probability on Age"}
plot_pdp(X_kfold, bst, feature_to_plot, feature_description, plot_params)

In [ ]:
feature_to_plot = "wbc"
feature_description = "White Blood Cell Count (10^9/L)"
plot_params = {"title": "Partial dependency of CRC probability on White Blood Cell Count"}
plot_pdp(X_kfold, bst, feature_to_plot, feature_description, plot_params)

In [ ]:
feature_to_plot = "alp"
feature_description = "Alkaline phosphatase (U/L)"
plot_params = {"title": "Partial dependency of CRC probability on Alkaline phosphatase (ALP)"}
plot_pdp(X_kfold, bst, feature_to_plot, feature_description, plot_params)

In [ ]:
feature_to_plot = "lym"
feature_description = "Lymphocyte percentage"
plot_params = {"title": "Partial dependency of CRC probability on Lymphocyte percentage"}
plot_pdp(X_kfold, bst, feature_to_plot, feature_description, plot_params)